# Citation Scrape
This notebook looks at obtaining the citations and their relations from UK Public General Acts (UKPGA).
<br><br>
This is achieved by loading in a `.csv` file containing all current acts under the category (as of 2022-06-27),
then using id of each document to access their official document hosted on [legislation.gov.uk](www.legislation.gov.uk).
Each document is stored in an `.xhtml` file format that follows the LegalDocML schema. Using this knowledge, scraping 
the hosted webpage, searching for `<a></a>` tags that contain `class="LegCitation"` will get every citation found within
the document.
<br><br>
We then look to labelling each citation into their respective relation to the current document. By looking at the
parent text surrounding the citation (the text before and after the citation), for the majority of citations, we can
identify the relationship. This can be achieved by looking for keywords for the relationship within the text, where a
match is found then the citation can be labelled with said relation.
<br><br>
Finally, all the cited documents and their citation relation graph is then stored in a csv file.

## Installs

In [1]:
!pip install requests
!pip install os
!pip install csv
!pip install pandas
!pip install bs4
!pip install re

You should consider upgrading via the 'D:\Documents\Projects\UK Citation Network\env\Scripts\python.exe -m pip install --upgrade pip' command.


## Imports

In [2]:
import requests
import os
import csv
import pandas as pd
from bs4 import BeautifulSoup
from re import search, IGNORECASE

## Opening the files

### File Paths

_These will need to be updated to match their path on your local system_

In [3]:
LEGISLATION_LIST_FILE_PATH = '# ENTER LEGISLATION LIST FILE PATH HERE'
LABEL_FILE_PATH = '# ENTER LABEL FILE PATH HERE'
CITATION_NETWORK_FILE_PATH = '# ENTER CITATION NETWORK FILE PATH HERE'

### List of Legislation

In [4]:
legislation_df = pd.read_csv(LEGISLATION_LIST_FILE_PATH)
legislation_list = zip(legislation_df['type'], legislation_df['year'], legislation_df['number'])

FileNotFoundError: [Errno 2] No such file or directory: '# ENTER LEGISLATION LIST FILE PATH HERE'

### List of Keywords / Labels

In [ ]:
def get_labels():
    with open(LABEL_FILE_PATH, encoding='utf-8') as f:
        # Read each line as a new entry into an array
        data = f.read().split('\n')
    return data

## Scraping

### Opening URL

In [ ]:
def get_from_url(doc_id):
    # Get the content from the legislation.gov.uk webpage and parse into BeautifulSoup
    
    url = f'https://www.legislation.gov.uk/{doc_id}/data.xht?view=snippet&wrap=true'
    r = requests.get(url)
    return BeautifulSoup(r.content)

### Extraction of Citations

In [ ]:
def get_citations(doc_soup):
    return doc_soup.find_all('a', 'LegCitation')

### Extraction of Parent Text

In [ ]:
def get_parent_text(citation_list):
    # Get the text surrounding the citation
    
    citation_passage_list = []
    for citation in citation_list:
        citation_passage_list.append(citation.parent.parent.text)
    return citation_passage_list

## Labelling of Citations

### Keyword Identification

In [ ]:
def label_citations(citations, parent_text):
    # For each citation, look at its parent text and label them according to the keyword found in the text
    
    labels = get_labels()
    citation_passage_label_list = []
    for i in range(len(citations)):
        passage = parent_text[i]
        for label in labels:
            if search(label, str(passage), IGNORECASE):
                citation_passage_label_list.append([label, citations[i].get('href')])
                break
    return citation_passage_label_list

### Generation of Valid URLs

In [ ]:
def get_data_urls(citations):
    """
    The representation of a citation's url can change from document to document. Sometimes it is a full url, sometimes
    it is a partial url, other times it can be a url to a webpage other than legislation.gov.uk. This function looks to
    standardise the url so the relevant details can be extracted easier.
    """
    urls = []
    for citation in citations:
        valid_citation = True
        citation_elements = citation.split('/')

        if citation_elements[0] != 'https:' and citation_elements[0] != 'http:':
            # Check to see if it is a full url, if not then make it a full url
            citation_elements = ('https://www.legislation.gov.uk' + citation).split('/')
        else:
             if citation_elements[2] == 'www.opsi.gov.uk':
                # Check to see if the url points to legislation.gov.uk, if not then make it
                citation_elements[2] = 'www.legislation.gov.uk'
                citation_elements.pop(3)

        if len(citation_elements) < 4:
            # If the citation is too short, declare it as an invalid citation
            valid_citation = False

        elif citation_elements[3] == 'id':
            citation_elements.pop(3)

        elif citation_elements[3] == 'european':
            if citation_elements[4] == 'regulation':
                citation_elements[3] = 'eur'
            elif citation_elements[4] == 'decision':
                citation_elements[3] = 'eudn'
            elif citation_elements[4] == 'directive':
                citation_elements[3] = 'eudr'
            else:
                # Raise an error if this type of url has never been seen
                raise ValueError(f'Not sure what to do for {"/".join(citation_elements)}')
            citation_elements.pop(4)

        elif citation_elements[3] == 'uksi' or citation_elements[3] == 'ukpga' or citation_elements[3] == 'ukcm' \
                or citation_elements[3] == 'ssi' or citation_elements[3] == 'nisr':
            pass

        elif citation_elements[3] == 'ukci' and '-' in citation_elements[4]:
            valid_citation = False

        else:
            # Raise an error if this type of url has never been seen
            raise ValueError(f'Not sure what to do for {"/".join(citation_elements)}')

        if valid_citation:
            citation_url = '/'.join(citation_elements)
            citation_url += '/data.xht?view=snippet&wrap=true'
            urls.append(citation_url)
    return urls

## Graph Generation

### Creation of CSV File

In [ ]:
def create_graph(header=None):
    if not (os.path.exists(CITATION_NETWORK_FILE_PATH)):
        # Check to see if graph exists, if not then initialise the file
        if header is None:
            header = ['source','target','relation']
        with open(CITATION_NETWORK_FILE_PATH, 'w') as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(header)

### Amending to CSV File

In [ ]:
def save_graph(doc_id, citation_urls, labels):
    TYPE = 3
    YEAR = 4
    NUMBER = 5
    with open(CITATION_NETWORK_FILE_PATH, 'a') as f:
        csv_writer = csv.writer(f)
        doc_id=doc_id.replace(',','')
        for i, url in enumerate(citation_urls):
            # Add the current document and the target document along with the label of the citation
            url = url.replace(',','')
            url_list = url.split('/')
            target_url = f'{url_list[TYPE]}_{url_list[YEAR]}_{url_list[NUMBER]}'
            row = [doc_id, target_url, labels[i]]
            csv_writer.writerow(row)

## Main Program

_This can take sometime to process as there is a lot of legislation to access and each act can contain **a lot of**
citations!_

In [ ]:
create_graph()

for file in legislation_list:
    # For each legislation in the list
    doc_id = f'{file[0].lower()}/{file[1]}/{file[2]}'
    try:
        print(f'Opening: {doc_id}')
        soup = get_from_url(doc_id) # Get the file
        citations = get_citations(soup) # Get the citations in the document
        print(f'    Found {len(citations)} citations in document')

        if len(citations) != 0:
            labelled_citations = label_citations(citations, get_parent_text(citations)) # Label the citations
            print(f'    Labelled {len(labelled_citations)} citations in document')

            if len(labelled_citations) != len(citations):
                print(f'    {len(citations) - len(labelled_citations)} citations not labelled')

            if len(labelled_citations) == 0:
                print(get_parent_text(citations))
            else:
                citation_urls = get_data_urls([labelled_citations[i][1] for i, _ in enumerate(labelled_citations)])
                labels = [labelled_citations[i][0] for i, _ in enumerate(labelled_citations)]
                save_graph(doc_id.replace('/','_'), citation_urls, labels) # Save the citation and the label to the network
                print(f'    Successfully saved citations to CSV file')
        print()

    except Exception as e:
        print(f'\n"{doc_id}" failed\n')
        e.with_traceback()
        break